In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import seaborn
import pandas as pd

In [2]:
#!pip install faiss-cpu numpy langchain-openai langchain-community sentence_transformers typing

In [3]:
import os
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from typing import List

In [18]:
os.getcwd()

'/Users/michaelaeichel/Documents/4940_Grad_Project'

In [20]:
simpletext_auto_documents = []
for st_file in os.listdir('./simpletext_auto'):
    text = open(f'/Users/michaelaeichel/Documents/4940_Grad_Project/simpletext_auto/{st_file}', "r")
    read_text = text.read()
    simpletext_auto_documents.append(read_text)

In [4]:
model_name = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [21]:
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [22]:
documents = simpletext_auto_documents

In [23]:
document_embeddings = embeddings_model.encode(documents, convert_to_tensor=True).cpu().numpy()
document_embeddings = np.array(document_embeddings).astype('float32')

In [24]:
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

In [25]:
class SimpleRetriever:
    def __init__(self, index, documents: List[str]):
        self.index = index
        self.documents = documents
        self.embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    def retrieve(self, query: str, top_k: int = 3) -> List[str]:
        query_embedding = self.embeddings_model.encode([query], convert_to_tensor=True).cpu().numpy().astype('float32')
        _, indices = self.index.search(query_embedding, top_k)
        return [self.documents[i] for i in indices[0]]

In [43]:
class LLMWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def invoke(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            **inputs,
            max_length=100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [36]:
class SimpleRAG:
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever

    def generate(self, query: str) -> str:
        retrieved_docs = self.retriever.retrieve(query)
        augmented_query = f"Question: {query}\nAnswer:"
        response = self.llm.invoke(augmented_query)
        return response

In [44]:
retriever = SimpleRetriever(index, documents)
llm = LLMWrapper(model, tokenizer)  # Using your existing model and tokenizer
rag = SimpleRAG(llm, retriever)

In [45]:
query = "what is the vascular depression hypothesis?"
response = rag.generate(query)
print(response)

Question: what is the vascular depression hypothesis?
Answer: the vascular depression hypothesis is that the disease is caused by a change in the vascular system. The vascular depression hypothesis is that changes in the vascular system result from changes in the structure of the blood vessels.
For example, it has been reported that the heart is one of the most important organ in the body's vascular system, resulting in a change in the structure of the heart.
Interestingly, it was also recently reported that the arterial
